In [7]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

with open("input_16.txt", 'r') as f:
    data = f.read()


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
#data = [
#    [int(match.group(1)), list(map(int, match.group(2).split()))]
#    for line in data.split("\n")
#    if (match := re.match(r'^(\d+):\s*(\d+(?:\s+\d+)*)', line))
#]
#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]

#grid, moves = data.split("\n\n")
#moves = moves.replace("\n", "")

grid = {(x,y): str(val) for y, r in enumerate(data.split('\n')) for x, val in enumerate(r)}
grid
#data = [int(x) for x in data.split(" ")]


# Output the parsed data
#print(data)

helpers.visualize_dict_grid(grid) 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# . # . . . . . . . . . # . # . . . # . . . # . . . . . . . . . . . # . . . . . # . . . . . . . # . . . # . . . . . . . . . . . # . . . . . . . . . . . . . # . . . . . # . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . E #
# . # . # . # # # # # . # . # . # . # . # . # . # # # # # # # . # . # . # # # . # # # . # . # # # . # . # . # # # . # # # # # # # . # # # # # # # . # # # . # . # # # . # . # # # . # . # # # # # # # . # # # . # . # . # . # . # . # . # # # . # # # # # # # # # # # . # # # # # . # . #
# . # . # . . . . . # . . . # . # . . . # . . . . . . . # . . . # . . . # . . . . . . . # . . . . . . . . . . . . . . . . . # . . . . . # . . . . . # . # 

In [ ]:
moves_dx = {"<": np.array([-1,0]),">": np.array([1,0]),"^": np.array([0,-1]),"v": np.array([0,1])} 

orig_location = [key for key, value in grid.items() if value == "S"][0]
visited = set()
lowest_found = 999999999999999999999999999999999999999999999999
def DFS(location, visited, heading, score):
    global lowest_found
    if tuple(location) in visited or score >= lowest_found:
        return
    #n_grid = grid.copy()
    #n_grid[tuple(orig_location)] = "."
    #n_grid[tuple(location)] = "@"
    #helpers.visualize_dict_grid(n_grid) 
    routes = []
    if grid[tuple(location)] == "E":
        if score < lowest_found:
            lowest_found = score
            print(lowest_found)
        return score
    visited.add(tuple(location))
    for new_heading, move in moves_dx.items():
        new_location = location + move
        if grid[tuple(new_location)] == "." or grid[tuple(new_location)] == "E":
            DFS_score = DFS(new_location, visited.copy(), new_heading, score + 1 + 1000 * (heading != new_heading))
            if DFS_score:
                routes.append(DFS_score)
    if routes:
        return min(routes)
    return 
DFS(orig_location, visited, ">", 0)